### Imports

In [1]:
%pip install torch
%pip install torchvision
%pip install phiflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from phi.torch.flow import *

### Loading data

In [3]:
## CHANGE CODE TO WORK FOR PHIFLOW, COPIED FROM PYTORCH EXAMPLES

class FluidDataset(Dataset):
        """Face Landmarks dataset."""

def __init__(self, csv_file, root_dir, transform=None):
    """
    Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.landmarks_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

def __len__(self):
    return len(self.landmarks_frame)

def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()

    img_name = os.path.join(self.root_dir,
                            self.landmarks_frame.iloc[idx, 0])
    image = io.imread(img_name)
    landmarks = self.landmarks_frame.iloc[idx, 1:]
    landmarks = np.array([landmarks])
    landmarks = landmarks.astype('float').reshape(-1, 2)
    sample = {'image': image, 'landmarks': landmarks}

    if self.transform:
        sample = self.transform(sample)

    return sample

NameError: name 'Dataset' is not defined

### Basic CNN

In [38]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # expected input is 32 x 32 x 3
        self.conv1 = nn.Conv2d(3, 6, 5)
        # size: 28 x 28 x 6
        self.pool = nn.MaxPool2d(2, 2)
        # size: 10 x 10 x 6
        self.conv2 = nn.Conv2d(6, 12, 5)
        # size: 5 x 5 x 12
        self.fc1 = nn.Linear(300, 3072)
        # ouput size: 3072
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

net = Net()

### Loss Function

In [23]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training the Network

In [39]:
bounds=Box[0:32, 0:32]
GRAVITY = math.tensor([0, -9.81])
DT = 1

water_ref = CenteredGrid(Noise(), extrapolation.BOUNDARY, x=32, y=32, bounds=bounds)
velocity_ref = StaggeredGrid(0, extrapolation.ZERO, x=32, y=32, bounds=bounds)

for epoch in range(10):  # loop over the dataset multiple times

    water = water_ref
    velocity = velocity_ref
    
    velocity += water * GRAVITY * DT @ velocity
    velocity, _ = fluid.make_incompressible(velocity)
    
    running_loss = 0.0
    for i in range(20):
        # get the inputs; data is a list of [inputs, labels]
        density = water.values.numpy(["x","y"])
        vel_x = velocity.at_centers().vector['x'].values.numpy(["x","y"])
        vel_y = velocity.at_centers().vector['y'].values.numpy(["x","y"])
        x = np.stack((density, vel_x, vel_y), axis=0)
        x = np.asarray([x])
        x_2 = torch.from_numpy(x)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        pred_y = net(x_2)
        loss = criterion(pred_y, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

NameError: name 'y' is not defined